In [12]:
import warnings
warnings.filterwarnings('ignore')

import sys


import pandas as pd
import numpy as np
from plotnine import *

from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_score

%matplotlib inline

# 0. Together

K-means is one of the most simple clustering algorithms out there. You randomly (or, in order to make convergence quicker, cleverly) choose K centroids in the feature space, you assign each data point to the centroid/cluster closest to it, and then recalculate the centroid by taking the mean (for each predictor) of all the data points in each cluster. 

This process iteratively repeats until either 1) cluster assignments don't change from step to step OR 2) the centroid doesn't change much from step to step.

<img src="https://drive.google.com/uc?export=view&id=1RHRfcPIjIZ_-IMOE00gyzVadlaGxXPh8" width=350px />

One thing to keep in mind with K-Means is that is assumes *spherical* variance within each cluster. That means that K-means behaves as if--within each cluster--all predictors have the same variance. Because of this, it is often good to either check this assumption, or z-score your variables so that they're on the same scale.

# 1. K-Means Function

Let's write our own simplified K-means function. Your function, `KM()` should take in two arguments:

- `df` a dataframe with all of your data.
- `k` the number of clusters to fit.

and apply K-Means to it. Remember that the steps of K means are:

**1**. Randomly select k centroids.
- I recommend choosing `k` random data points from `df`. You can do this by using `np.random.choice(range(0,df.shape[0]), k)` to select the indices for `k` randomly selected rows. THEN use those indices to grab the chosen rows from df and store them.
    
**2**. Assign each data point from `df` to the closest centroid.
- You'll need to calculate the distance between each data point and each centroid. Perhaps look at the KNN classwork to see how to do that using `np.linalg.norm()` (see Hint 3).
    - I recommend storing cluster/centroid membership by having a dictionary with one key for each cluster/centroid, and the value is a list of row indices pertaining to the data points in each cluster (see HINT 1 for an example of this.
    
**3**. Re-calculate the cluster mean/centroid
- For each centroid/cluster, find the mean value for each predictor/feature by taking the mean for that feature from all the data points assigned to the centroid/cluster.(see Hint 2)
    
**4**. Repeat Steps 2-3 until the change in centroid positions are all less than 0.0001
- in other words, calculate how far each centroid moved. If all of them moved less than 0.0001 units, then stop.
    
**5**. Return the cluster assignments by returning the dictonary of the clusters and their memberships that you create in #2.

### HINT 1:

Store your cluster memberships like this (in this case k = 3, and there are only 20 datapoints, but your function should take any k, and any number of data points):

```
clust = {"0": [0,7,4,5,12,18,20],
          "1": [10,8,3,2,14,17,19],
          "2": [1,6,7,9,11,13,15,16]}
```
      

### HINT 2:

If a cluster contained the following data points:

|           | X1 | X2 | X3 |
|-----------|----|----|----|
| Person 1  | 5  | 2  | 9  |
| Person 2  | 2  | 3  | 2  |
| Person 3  | 1  | 6  | 1  |
| Person 4  | 7  | 1  | 4  |
| Person 5  | 3  | 2  | 5  |
| Person 6  | 1  | 1  | 8  |
| Person 7  | 7  | 0  | 6  |
| Person 8  | 0  | 7  | 2  |
| Person 9  | 2  | 3  | 7  |
| Person 10 | 4  | 6  | 1  |


Then the centroid for that cluster would be [a,b,c] where a, b, and c are the means of each column X1, X2, and X3:

a = (5 + 2 + 1 + 7 + 3 + 1 + 7 + 0 + 2 + 4)/10 

b = (2 + 3 + 6 + 1 + 2 + 1 + 0 + 7 + 3 + 6)/10

c = (9 + 2 + 1 + 4 + 5 + 8 + 6 + 2 + 7 + 1)/10




### HINT 3:

To calculate the distance between two vectors, you can use:

```
distance_ab = np.linalg.norm(a-b)

```

where `a` and `b` are the two vectors.

In [42]:
def KM(df,k):
    
    # 1. randomly select k centroids
    
    ### YOUR CODE HERE ###-----------------------------------
    
    centroids = []
    
    for i in range(0,k):
        centroids.append(df.iloc[np.random.choice(range(i,df.shape[0], k))])
    
    
    print(centroids)
    
    ### /YOUR CODE HERE ###----------------------------------
    
    
    # 2. iterate through all the rows in df, and assign them to the
    # centroid closest to them, store those assignments in a dict
    
    closest = []
    
#     for i in range(0, df.shape[0]):
#         distance = sys.maxsize
#         center = 0
#         indx = 0
#         dfI = df.iloc[i]
#         for j in centroids:
#             distance_ij = abs(np.linalg.norm(dfI-j))
#             if distance_ij < distance:
#                 center = indx
#                 distance = distance_ij
#             indx = indx + 1
            
#         closest.append(center)
    
#     print(closest)

    converged = False # has the algorithm converged yet?
    
    while not converged: # until the centroids stop moving
        
        # add all the dictionary keys for each cluster
        clust = {}
        for c in range(0,k):
            clust[str(c)] = []
            

        for dataPoint in range(0, df.shape[0]):

            # calculate distances between dataPoint and each centroid
            
            ### YOUR CODE HERE ###-----------------------------------
            
            dfI = df.iloc[dataPoint]
            distances = []
            
            for c in range(0,k):
                distances.append(abs(np.linalg.norm(dfI-c)))
    
            ### /YOUR CODE HERE ###----------------------------------

            # find the centroid that's closest
            # hint: use np.argmin()
            
            clust[str(np.argmin(distances))].append(dataPoint)
            
            ### YOUR CODE HERE ###-----------------------------------
            converged = True
    
            ### /YOUR CODE HERE ###----------------------------------

            # add dataPoint to that cluster in the clust dictionary
            
            ### YOUR CODE HERE ###-----------------------------------
    
    
            ### /YOUR CODE HERE ###----------------------------------

        # 3. re-calculate the center/centroid of each cluster

        #new_centroids = [[] for c in range(0,k)] #create an empty list of k lists to fill in later

        #for c in range(0,k): 
            # for each cluster calculate the new centroid values
            # hint: remember the new centroid value is the mean
            # (for each predicor) of data points in that cluster

            ### YOUR CODE HERE ###-----------------------------------
            # calculate the new centroids and add them to new_centroids
            
    
            ### /YOUR CODE HERE ###----------------------------------

        #new_centroids = np.array(new_centroids) # change new centroids to be an array so that #4 works

        # 4. check whether you can stop iterating by checking whether the
        # distance between the previous position and current position is
        # less than 0.0001 for all k centroids.

        # calculate the distance between the old centroid values, and new_centroids values
        #change = np.array([np.linalg.norm(centroids[i]-new_centroids[i]) for i in range(0,k)])
        
        # check whether all of them moved less than 0.0001 units.
        #converged = np.all(change < 0.0001)

        # set new_centroids to be established centroids
        #centroids = new_centroids


    # 5. Return cluster memberships dictionary
    print(clust)
    return("done")

In [43]:
data = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/programmers3.csv")

data.head()

# if you want to you can test your function on data!

KM(data, 4)

[py    31.863162
r     20.673834
Name: 184, dtype: float64, py    53.015136
r     16.216788
Name: 205, dtype: float64, py    56.570726
r     33.215489
Name: 210, dtype: float64, py    88.022476
r     80.529736
Name: 87, dtype: float64]
{'0': [], '1': [], '2': [], '3': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 

'done'

# 2. Using your K-Means Function

Now that you have done the incredibly impressive work of writing (with some help!) your own K-means function. Let's use it and compare the results to what we'd get from `sklearn`!

First, use your OWN function `KM()` to do K-means on `data` with k = 5. Then generate the cluster assingments using the code provided. Then make a ggplot scatterplot of your clusters.

Second, use sklearn's `KMeans()` function to do K-means on `data` with k = 5. Then generate the cluster assignments using `.predict()`. Then make a ggplot scatterplot of your clusters.

In [ ]:
# USING YOUR FUNCTION-----

# run k-means using YOUR function

### YOUR CODE HERE ###-----------------------------------
clusters = ###
### /YOUR CODE HERE ###----------------------------------

# generate assignments
assignments = np.array([999 for row in range(0, data.shape[0])])

for cluster in clusters:
    assignments[clusters[cluster]] = cluster
    
data["assignments_ME"] = assignments

# create ggplot scatter plot of data, using x, y and color = "assignments_ME"
###

In [ ]:
# USING SKLEARN

# create kmeans model

### YOUR CODE HERE ###-----------------------------------

### /YOUR CODE HERE ###----------------------------------

# fit kmeans model

### YOUR CODE HERE ###-----------------------------------

### /YOUR CODE HERE ###----------------------------------

# get assignments

### YOUR CODE HERE ###-----------------------------------

assignments_SK = ###

### /YOUR CODE HERE ###----------------------------------


# add assignments to data
data["assignments_SK"] = assignments_SK

# create another ggplot scatter plot of data, using x, y and color = "assignments_SK"
### YOUR CODE HERE ###-----------------------------------

### /YOUR CODE HERE ###----------------------------------

### *Question*

How do your results compare?
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" width = 200px />